In [1]:
import pandas as pd
from litellm import completion
import litellm
import os
litellm.enable_json_schema_validation = True
litellm.set_verbose = True
import json
from litellm import ModelResponse
from dotenv import load_dotenv
load_dotenv()

df = pd.read_json("hf://datasets/HuggingFaceH4/MATH-500/test.jsonl", lines=True)

In [2]:
df.head(2)

,problem,solution,answer,subject,level,unique_id
0,"Convert the point $(0,3)$ in rectangular coord...",We have that $r = \sqrt{0^2 + 3^2} = 3.$ Also...,"\left( 3, \frac{\pi}{2} \right)",Precalculus,2,test/precalculus/807.json
1,Define\n\[p = \sum_{k = 1}^\infty \frac{1}{k^2...,We count the number of times $\frac{1}{n^3}$ a...,p - q,Intermediate Algebra,5,test/intermediate_algebra/1994.json


In [3]:
from pydantic import BaseModel
class Answer(BaseModel):
  reasoning: str
  answer: str

class AnswerCorrectness(BaseModel):
  correct:bool

response_schema_answer = {
    "type": "object",
    "properties": {
        "reasoning": {
            "type": "string"
        },
        "answer": {
            "type": "string"
        }
    },
    "required": ["reasoning", "answer"]
}


In [6]:
def process_math_problems(df, start_idx:int, end_idx:int):
    for idx in range(start_idx, min(len(df), end_idx + 1)):
        response = completion(model='gemini/gemini-2.0-flash', messages=[
            {
                'role': 'system',
                'content': '''Be a helpful assistant.
                You need to just give me the final answer and no other text. Don't tell the steps. Just give the final output for the answer key 
                and your reasoning in the reasoning key. 
                
                example:
                user query: What is the area of a rectangle with length 3cm and breadth 4cm. 
                assistant output: 
                {
                    "reasoning": "area of a rectangle is length * breadth, so here it will be 3cm*4cm which is 12cm squared."
                    "answer" : "area is 12 cm squared."
                }
                ''',
            },
            {
                'role': 'user',
                'content': df.iloc[idx]['problem'],
            }],
            response_format={"type": "json_object", "response_schema": response_schema_answer}
        )
        answer_content = response.choices[0]['message']['content']

        answer_obj = Answer.model_validate_json(answer_content)
        answer = answer_obj.answer

        response = completion(model='gpt-4o', messages=[
            {
                'role': 'system',
                'content': '''You are an intellegent maths professor. I will give you 2 answers. 
                    One Model answer and one student answer. You whether the answer is right or wrong.Return a json with key are correct and value as True or False depeding on your evaluation''',
            },
            {
                'role': 'user',
                'content': f'Model Answer : {df.iloc[idx]["answer"]}, Student Answer : {answer}',
            },
        ], 
        response_format=AnswerCorrectness)
        
        answer_correctness = response.choices[0]['message']['content']
        answer_correctness_obj = AnswerCorrectness.model_validate_json(answer_correctness)
        
        df.at[idx, 'llm_raw_response'] = answer_content
        df.at[idx, 'llm_answer'] = answer
        df.at[idx, 'is_correct'] = answer_correctness_obj.correct
        
        print(f"Processed row {idx}")


In [ ]:
process_math_problems(df,start_idx=0,end_idx=50)

In [8]:
df.to_csv("50 rows gemini.csv")

In [19]:
def generate_training_messages(df, start_idx=0, num_rows=30):
    end_idx = min(start_idx + num_rows, len(df))
    messages = [{
                'role': 'system',
                'content': '''Be a helpful assistant.
                You need to just give me the final answer and no other text. Don't tell the steps. Just give the final output for the answer key 
                and your reasoning in the reasoning key. 
                
                example:
                user query: What is the area of a rectangle with length 3cm and breadth 4cm. 
                assistant output: 
                {
                    "reasoning": "area of a rectangle is length * breadth, so here it will be 3cm*4cm which is 12cm squared."
                    "answer" : "area is 12 cm squared."
                }
                ''',
            }]
    for idx in range(start_idx, end_idx):
        row = df.iloc[idx]
        
        # Base messages that are common for all examples
        messages.extend([
            {
                'role': 'user',
                'content': row['problem']
            },
            {
                'role': 'assistant',
                'content': row['llm_raw_response']
            }
        ])
        
        # Add feedback based on correctness
        if not row['is_correct']:
            messages.append({
                'role': 'user',
                'content': f"Let me correct this. The right answer is {row['answer']}. Let's understand the solution: {row['solution']}"
            })
        else:
            messages.append({
                'role': 'user',
                'content': "Good job! Your reasoning and answer are correct!"
            })
        messages.append({
            'role': 'assistant',
            'content': "Understood. I will keep this in mind"
        })
            
       
    
    return messages

In [20]:
training_data = generate_training_messages(df)

In [21]:
len(training_data)

121

In [22]:
def test_process_math_problems_te(df, start_idx:int, end_idx:int):
    for idx in range(start_idx, min(len(df), end_idx + 1)):
        messages = training_data + [
            {
                'role': 'system',
                'content': '''Be a helpful assistant.
                You need to just give me the final answer and no other text. Don't tell the steps. Just give the final output for the answer key 
                and your reasoning in the reasoning key. 
                
                example:
                user query: What is the area of a rectangle with length 3cm and breadth 4cm. 
                assistant output: 
                {
                    "reasoning": "area of a rectangle is length * breadth, so here it will be 3cm*4cm which is 12cm squared."
                    "answer" : "area is 12 cm squared."
                }
                ''',
            },
            {
                'role': 'user',
                'content': df.iloc[idx]['problem'],
            }]
        
        response = completion(model='gemini/gemini-2.0-flash', messages=messages,
            response_format={"type": "json_object", "response_schema": response_schema_answer}
        )
        answer_content = response.choices[0]['message']['content']

        answer_obj = Answer.model_validate_json(answer_content)
        answer = answer_obj.answer

        response = completion(model='gpt-4o', messages=[
            {
                'role': 'system',
                'content': '''You are an intellegent maths professor. I will give you 2 answers. 
                    One Model answer and one student answer. You whether the answer is right or wrong.Return a json with key are correct and value as True or False depeding on your evaluation''',
            },
            {
                'role': 'user',
                'content': f'Model Answer : {df.iloc[idx]["answer"]}, Student Answer : {answer}',
            },
        ], 
        response_format=AnswerCorrectness)
        
        answer_correctness = response.choices[0]['message']['content']
        answer_correctness_obj = AnswerCorrectness.model_validate_json(answer_correctness)
        
        df.at[idx, 'llm_raw_response_test'] = answer_content
        df.at[idx, 'llm_answer_test'] = answer
        df.at[idx, 'is_correct_test'] = answer_correctness_obj.correct
        
        print(f"Processed row {idx}")


In [23]:
test_process_math_problems_te(df,start_idx=31,end_idx=50)

00:53:23 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'response_mime_type': 'application/json', 'response_schema': {'type': 'object', 'properties': {'reasoning': {'type': 'string'}, 'answer': {'type': 'string'}}, 'required': ['reasoning', 'answer']}}


00:53:27 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


RAW RESPONSE:
{
  "candidates": [
    {
      "content": {
        "parts": [
          {
            "text": "{\n  \"answer\": \"11\\\\sqrt{2}\",\n  \"reasoning\": \"We can simplify $\\\\sqrt{242}$ by factoring out perfect squares.  Since $242 = 121 \\n\\\\cdot 2 = 11^2 \\\\cdot 2,$ $\\\\sqrt{242} = \\\\sqrt{11^2 \\\\cdot 2} = \\\\sqrt{11^2} \\\\cdot \\\\sqrt{2} = 11\\\\sqrt{2}.$\"\n}"
          }
        ],
        "role": "model"
      },
      "finishReason": "STOP",
      "avgLogprobs": -0.15258037732995075
    }
  ],
  "usageMetadata": {
    "promptTokenCount": 32004,
    "candidatesTokenCount": 115,
    "totalTokenCount": 32119,
    "promptTokensDetails": [
      {
        "modality": "TEXT",
        "tokenCount": 32004
      }
    ],
    "candidatesTokensDetails": [
      {
        "modality": "TEXT",
        "tokenCount": 115
      }
    ]
  },
  "modelVersion": "gemini-2.0-flash"
}



SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Fin

00:53:28 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


...

RAW RESPONSE:
{
  "candidates": [
    {
      "content": {
        "parts": [
          {
            "text": "{\n  \"answer\": \"-83\",\n  \"reasoning\": \"We translate the given information into the equation \\\\frac{1}{2}(3x-9) = x+37. Multiplying both sides by 2, we obtain 3x-9 = 2x+74. Subtracting 2x from both sides, we get x-9 = 74. Adding 9 to both sides gives x = 83. Therefore, the value of x is -83.\"\n}"
          }
        ],
        "role": "model"
      },
      "finishReason": "STOP",
      "avgLogprobs": -0.1723625659942627
    }
  ],
  "usageMetadata": {
    "promptTokenCount": 32021,
    "candidatesTokenCount": 112,
    "totalTokenCount": 32133,
    "promptTokensDetails": [
      {
        "modality": "TEXT",
        "tokenCount": 32021
      }
    ],
    "candidatesTokensDetails": [
      {
        "modality": "TEXT",
        "tokenCount": 112
      }
    ]
  },
  "modelVersion": "gemini-2.0-flash"
}



SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cac

00:55:09 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


RAW RESPONSE:
{"id": "chatcmpl-B1eYGd6xKZaHrl4yeYoeIFixvBEya", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "{\"correct\":false}", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1739733908, "model": "gpt-4o-2024-08-06", "object": "chat.completion", "service_tier": "default", "system_fingerprint": "fp_523b9b6e5f", "usage": {"completion_tokens": 6, "prompt_tokens": 114, "total_tokens": 120, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details": {"audio_tokens": 0, "cached_tokens": 0}}}


Processed row 49
SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'response_mime_type': 'application/json', 'response_schema': {'type': 'object', 'properties': {'reasoning': {'type': 'string'}, 'answer': {'type': 'stri

00:55:16 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


RAW RESPONSE:
{
  "candidates": [
    {
      "content": {
        "parts": [
          {
            "text": "{\n  \"answer\": \"247\",\n  \"reasoning\": \"Let $x$ be the number of coins in each of the seven bags. After finding the bag of 53 coins, the total number of coins is $7x + 53$. We redistribute the coins so that each of the 8 bags has the same number of coins. Let $y$ be the number of coins in each of the 8 bags. Then $8y = 7x + 53$. Also, we know that $7x \u003e 200$, so $x \u003e \\\\frac{200}{7} \\\\approx 28.57$. We are looking for the smallest possible integer value for $x$. We need to find $x$ such that $7x + 53$ is divisible by 8. We can rewrite the equation as $7x + 53 \\\\equiv 0 \\\\pmod{8}$. Since $7 \\\\equiv -1 \\\\pmod{8}$ and $53 \\\\equiv 5 \\\\pmod{8}$, we have $-x + 5 \\\\equiv 0 \\\\pmod{8}$, which means $x \\\\equiv 5 \\\\pmod{8}$. So $x$ must be of the form $8k + 5$ for some integer $k$. Since $x \u003e 28.57$, we must have $8k + 5 \u003e 28.57$, so $8k \

In [15]:
# df.to_csv('first 50 test gemini.csv')

In [27]:
df_test = df.iloc[31:50]

In [28]:
df_test['is_correct'].value_counts()

is_correct
True     10
False     9
Name: count, dtype: int64

In [29]:
df_test['is_correct_test'].value_counts()

is_correct_test
False    10
True      9
Name: count, dtype: int64